In [2]:
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import json

In [3]:
import dataset
import algorithm

In [4]:
CONFIG = json.load(open('config.json'))

In [5]:
Dataset = dataset.download_dataset(CONFIG['data']['filename'])

In [6]:
Dataset.head()

,area,material,condicion,anio_construccion,anio_remodelacion,sotano,calefaccion,aire_acondicionado,area_construida_piso_1,area_construida_piso_2,...,banios,banios_sin_ducha,dormitorios,cocinas,chimeneas,area_garage,area_piscina,mes_venta,anio_venta,precio
0,8450,7,5,2003,2003,Gd,GasA,Y,856,854,...,2,1,3,1,0,548,0,2,2008,208500
1,9600,6,8,1976,1976,Gd,GasA,Y,1262,0,...,2,0,3,1,1,460,0,5,2007,181500
2,11250,7,5,2001,2002,Gd,GasA,Y,920,866,...,2,1,3,1,1,608,0,9,2008,223500
3,9550,7,5,1915,1970,TA,GasA,Y,961,756,...,1,0,3,1,1,642,0,2,2006,140000
4,14260,8,5,2000,2000,Gd,GasA,Y,1145,1053,...,2,1,4,1,1,836,0,12,2008,250000


In [7]:
data = dataset.get_data(Dataset)
data.head()

,area,material,condicion,anio_construccion,anio_remodelacion,sotano,calefaccion,aire_acondicionado,area_construida_piso_1,area_construida_piso_2,...,banios,banios_sin_ducha,dormitorios,cocinas,chimeneas,area_garage,area_piscina,mes_venta,anio_venta,precio
0,8450,7,5,2003,2003,0,1,1,856,854,...,2,1,3,1,0,548,0,2,2008,208500
1,9600,6,8,1976,1976,0,1,1,1262,0,...,2,0,3,1,1,460,0,5,2007,181500
2,11250,7,5,2001,2002,0,1,1,920,866,...,2,1,3,1,1,608,0,9,2008,223500
3,9550,7,5,1915,1970,1,1,1,961,756,...,1,0,3,1,1,642,0,2,2006,140000
4,14260,8,5,2000,2000,0,1,1,1145,1053,...,2,1,4,1,1,836,0,12,2008,250000


In [8]:
normalize_data = dataset.Normalizer.transform(data)

In [9]:
data_x, data_y = dataset.split_data_x_y(normalize_data)

In [10]:
data_x_train, data_x_test = dataset.split_data_train_test(data_x, CONFIG["data"]["train_split_size"])
data_y_train, data_y_test = dataset.split_data_train_test(data_y, CONFIG["data"]["train_split_size"])

In [11]:
dataset_train = dataset.TimeSeriesDataset(data_x_train, data_y_train)
dataset_test = dataset.TimeSeriesDataset(data_x_test, data_y_test)

In [12]:
# DataLoader => Devuelve un conjunto de iteraciones, cada iteracion contiene un arreglo de 'batch_size' registros
train_dataloader = DataLoader(dataset_train, CONFIG["training"]["batch_size"], shuffle=True)
test_dataloader = DataLoader(dataset_test, CONFIG["training"]["batch_size"], shuffle=True)

In [13]:
model = algorithm.Model(config_model=CONFIG["model"], config_training=CONFIG["training"])

In [14]:
# Entrenamiento del modelo
for epoch in range(CONFIG["training"]["num_epoch"]):
    loss_train, lr_train = model.run_epoch(train_dataloader, is_training=True)
    loss_test, lr_test = model.run_epoch(test_dataloader)
    if epoch == 0 or ((epoch+1)%10 == 0):
        print('Epoch[{}/{}] | loss train:{:.6f}, test:{:.6f} | lr:{:.6f}'
              .format(epoch+1, CONFIG["training"]["num_epoch"], loss_train, loss_test, lr_train))

Epoch[1/100] | loss train:0.005892, test:0.001145 | lr:0.010000
Epoch[10/100] | loss train:0.001689, test:0.000475 | lr:0.010000
Epoch[20/100] | loss train:0.000999, test:0.000409 | lr:0.010000
Epoch[30/100] | loss train:0.000651, test:0.000484 | lr:0.010000
Epoch[40/100] | loss train:0.000800, test:0.000660 | lr:0.010000
Epoch[50/100] | loss train:0.000492, test:0.000464 | lr:0.010000
Epoch[60/100] | loss train:0.000496, test:0.000325 | lr:0.010000
Epoch[70/100] | loss train:0.000455, test:0.000307 | lr:0.010000
Epoch[80/100] | loss train:0.000449, test:0.000692 | lr:0.010000
Epoch[90/100] | loss train:0.000306, test:0.000674 | lr:0.010000
Epoch[100/100] | loss train:0.000281, test:0.000361 | lr:0.010000


In [19]:
# Verificacion de la prediccion del modelo
error = 0
i=0
for idx, (x, y) in enumerate(test_dataloader):
    i+=1
    x = x.to(CONFIG["training"]["device"])
    out = model(x)
    out_vector = out.detach().numpy()
    y_vector = y.numpy()
    for i in range(len(y_vector)):
        price_predict = dataset.Normalizer.inverse_transform(out_vector[i], 21)
        price_real = dataset.Normalizer.inverse_transform(y_vector[i], 21)
        porcentage = price_predict/price_real
        error += abs(1-porcentage)
        print('Price predict: {0}, Price real: {1}, Diferencia: {2} ({3})'.format(price_predict, price_real, price_predict-price_real, (porcentage)))
    
    print("margen de error {}%".format(round(100 * error / i, 2)))
    break

Price predict: 161635, Price real: 157000, Diferencia: 4635 (1.0295222929936305)
Price predict: 119969, Price real: 132500, Diferencia: -12531 (0.9054264150943396)
Price predict: 107846, Price real: 91500, Diferencia: 16346 (1.1786448087431693)
Price predict: 160072, Price real: 155835, Diferencia: 4237 (1.0271890140212405)
Price predict: 318099, Price real: 302000, Diferencia: 16099 (1.0533079470198676)
Price predict: 148938, Price real: 155000, Diferencia: -6062 (0.9608903225806451)
Price predict: 86412, Price real: 79000, Diferencia: 7412 (1.0938227848101265)
Price predict: 192828, Price real: 239000, Diferencia: -46172 (0.8068117154811716)
Price predict: 197163, Price real: 178000, Diferencia: 19163 (1.1076573033707866)
Price predict: 134696, Price real: 144000, Diferencia: -9304 (0.9353888888888889)
Price predict: 188999, Price real: 186500, Diferencia: 2499 (1.0133994638069705)
Price predict: 161977, Price real: 186700, Diferencia: -24723 (0.8675790037493305)
Price predict: 10191